# GOLD LAYER - RELATÓRIOS COM IDENTIFICAÇÃO MASCARADA

In [0]:
from pyspark.sql import DataFrame
from pyspark.sql.functions import *
from pyspark.sql.window import Window
from delta.tables import DeltaTable
from datetime import datetime


# CONFIGURAÇÃO

In [0]:
# Paths
STORAGE_ACCOUNT = "mystoacc"
GOLD_PATH = f"abfss://gold@{STORAGE_ACCOUNT}.dfs.core.windows.net"

# Catalog e Database
spark.sql("USE CATALOG hive_metastore")
spark.sql("USE healthcare_gold")

# Timestamp de processamento
PROCESSING_TIMESTAMP = datetime.now()
PROCESSING_DATE = PROCESSING_TIMESTAMP.strftime("%Y-%m-%d")

print(f"Processamento Gold - Relatórios com Identificação iniciado: {PROCESSING_TIMESTAMP}")
print("Usando dados mascarados (LGPD compliant)")
print("="*80)

# FUNÇÕES AUXILIARES

In [0]:
def add_gold_metadata(df: DataFrame) -> DataFrame:
    """Adiciona colunas de metadados da Gold"""
    return df.withColumn("gold_processed_at", lit(PROCESSING_TIMESTAMP)) \
             .withColumn("gold_processing_date", lit(PROCESSING_DATE))

def log_aggregation(table_name: str, record_count: int):
    """Loga métricas de agregação"""
    print(f"\n{table_name}:")
    print(f"  Registros agregados: {record_count:,}")

def save_to_gold(df: DataFrame, table_name: str) -> dict:
    """Salva DataFrame na Gold com modo OVERWRITE"""
    try:
        gold_path = f"{GOLD_PATH}/{table_name}"
        
        # Adicionar metadados
        df_final = add_gold_metadata(df)
        
        # Contar registros
        record_count = df_final.count()
        log_aggregation(table_name, record_count)
        
        # Escrever
        df_final.write.format("delta") \
            .mode("overwrite") \
            .option("overwriteSchema", "true") \
            .saveAsTable(f"healthcare_gold.{table_name}")
        
        print(f"✓ Tabela {table_name} salva na Gold")
        
        return {
            "table": table_name,
            "status": "SUCCESS",
            "records": record_count
        }
        
    except Exception as e:
        print(f"✗ ERRO ao salvar {table_name}: {str(e)}")
        return {
            "table": table_name,
            "status": "FAILED",
            "error": str(e)
        }


In [0]:
# CARREGAR DADOS

In [0]:
print("Carregando dados da Silver...")

# Fato e dimensões
df_consultas = spark.table("healthcare_silver.fato_consultas")
df_medico = spark.table("healthcare_silver.dim_medico")
df_paciente = spark.table("healthcare_silver.dim_paciente")
df_clinica = spark.table("healthcare_silver.dim_clinica")

# Views mascaradas (PII protegida)
df_medico_masked = spark.table("healthcare_silver.v_medico_identidade_masked")
df_paciente_masked = spark.table("healthcare_silver.v_paciente_identidade_masked")

print("✓ Dados carregados (incluindo views mascaradas)")

## RELATÓRIO 1: TOP MÉDICOS POR RECEITA (COM IDENTIFICAÇÃO MASCARADA)

In [0]:
print("\n" + "="*60)
print("Criando: rpt_top_medicos_receita")
print("="*60)

rpt_top_medicos = df_consultas.alias("f") \
    .join(df_medico.alias("m"), col("f.sk_medico") == col("m.sk_medico")) \
    .join(df_medico_masked.alias("mm"), col("f.sk_medico") == col("mm.sk_medico")) \
    .groupBy(
        col("m.sk_medico"),
        col("mm.nome_medico").alias("nome_medico_masked"),
        col("mm.crm").alias("crm_masked"),
        col("m.especialidade"),
        col("m.uf_crm")
    ) \
    .agg(
        count("f.id_consulta").alias("total_consultas"),
        sum("f.valor_total").alias("receita_total"),
        avg("f.valor_total").alias("ticket_medio"),
        countDistinct("f.sk_paciente").alias("pacientes_atendidos")
    ) \
    .withColumn("ranking", dense_rank().over(Window.orderBy(col("receita_total").desc()))) \
    .orderBy("ranking")

result_top_medicos = save_to_gold(rpt_top_medicos, "rpt_top_medicos_receita")


## RELATÓRIO 2: PACIENTES COM MAIOR GASTO (COM IDENTIFICAÇÃO MASCARADA)

In [0]:
print("\n" + "="*60)
print("Criando: rpt_pacientes_alto_gasto")
print("="*60)

rpt_pacientes_gasto = df_consultas.alias("f") \
    .join(df_paciente.alias("p"), col("f.sk_paciente") == col("p.sk_paciente")) \
    .join(df_paciente_masked.alias("pm"), col("f.sk_paciente") == col("pm.sk_paciente")) \
    .groupBy(
        col("p.sk_paciente"),
        col("pm.nome_completo").alias("nome_paciente_masked"),
        col("pm.email").alias("email_masked"),
        col("p.sexo"),
        col("p.cidade")
    ) \
    .agg(
        count("f.id_consulta").alias("total_consultas"),
        sum("f.valor_total").alias("gasto_total"),
        avg("f.valor_total").alias("gasto_medio"),
        max("f.valor_total").alias("maior_consulta")
    ) \
    .withColumn("ranking", dense_rank().over(Window.orderBy(col("gasto_total").desc()))) \
    .filter(col("ranking") <= 100) \
    .orderBy("ranking")

result_pacientes_gasto = save_to_gold(rpt_pacientes_gasto, "rpt_pacientes_alto_gasto")

In [0]:
## RELATÓRIO 3: MÉDICOS POR CLÍNICA (COM IDENTIFICAÇÃO)

In [0]:
print("\n" + "="*60)
print("Criando: rpt_medicos_por_clinica")
print("="*60)

rpt_medicos_clinica = df_consultas.alias("f") \
    .join(df_clinica.alias("c"), col("f.sk_clinica") == col("c.sk_clinica")) \
    .join(df_medico.alias("m"), col("f.sk_medico") == col("m.sk_medico")) \
    .join(df_medico_masked.alias("mm"), col("f.sk_medico") == col("mm.sk_medico")) \
    .groupBy(
        col("c.sk_clinica"),
        col("c.tipo_clinica"),
        col("c.estado"),
        col("m.sk_medico"),
        col("mm.nome_medico").alias("nome_medico_masked"),
        col("m.especialidade")
    ) \
    .agg(
        count("f.id_consulta").alias("consultas_realizadas"),
        sum("f.valor_total").alias("receita_gerada"),
        countDistinct("f.sk_paciente").alias("pacientes_unicos")
    ) \
    .orderBy("tipo_clinica", col("receita_gerada").desc())

result_medicos_clinica = save_to_gold(rpt_medicos_clinica, "rpt_medicos_por_clinica")

In [0]:
## RELATÓRIO 4: HISTÓRICO DE CONSULTAS POR PACIENTE (ÚLTIMAS 10)

In [0]:
print("\n" + "="*60)
print("Criando: rpt_historico_pacientes")
print("="*60)

rpt_historico = df_consultas.alias("f") \
    .join(df_paciente_masked.alias("pm"), col("f.sk_paciente") == col("pm.sk_paciente")) \
    .join(df_medico_masked.alias("mm"), col("f.sk_medico") == col("mm.sk_medico")) \
    .join(df_clinica.alias("c"), col("f.sk_clinica") == col("c.sk_clinica")) \
    .select(
        col("pm.nome_completo").alias("paciente_masked"),
        col("pm.email").alias("email_masked"),
        col("mm.nome_medico").alias("medico_masked"),
        col("c.tipo_clinica"),
        col("c.estado"),
        col("f.valor_total"),
        col("f.plano_cobriu")
    ) \
    .withColumn("row_num", row_number().over(
        Window.partitionBy("paciente_masked").orderBy(col("valor_total").desc())
    )) \
    .filter(col("row_num") <= 10) \
    .drop("row_num")

result_historico = save_to_gold(rpt_historico, "rpt_historico_pacientes")

In [0]:
## RELATÓRIO 5: MÉDICOS E PACIENTES POR CIDADE

In [0]:
print("\n" + "="*60)
print("Criando: rpt_medicos_pacientes_cidade")
print("="*60)

# Agregação de médicos por cidade (via clínica)
medicos_cidade = df_consultas.alias("f") \
    .join(df_clinica.alias("c"), col("f.sk_clinica") == col("c.sk_clinica")) \
    .join(df_medico_masked.alias("mm"), col("f.sk_medico") == col("mm.sk_medico")) \
    .groupBy(
        col("c.estado"),
        col("mm.nome_medico").alias("medico_masked")
    ) \
    .agg(
        count("f.id_consulta").alias("consultas"),
        sum("f.valor_total").alias("receita")
    )

# Agregação de pacientes por cidade
pacientes_cidade = df_consultas.alias("f") \
    .join(df_paciente.alias("p"), col("f.sk_paciente") == col("p.sk_paciente")) \
    .join(df_paciente_masked.alias("pm"), col("f.sk_paciente") == col("pm.sk_paciente")) \
    .groupBy(
        col("p.cidade"),
        col("pm.nome_completo").alias("paciente_masked")
    ) \
    .agg(
        count("f.id_consulta").alias("consultas"),
        sum("f.valor_total").alias("gasto")
    )

# União das duas agregações
rpt_cidade = medicos_cidade.union(
    pacientes_cidade.select(
        col("cidade").alias("estado"),
        col("paciente_masked").alias("medico_masked"),
        col("consultas"),
        col("gasto").alias("receita")
    )
)

result_cidade = save_to_gold(rpt_cidade, "rpt_medicos_pacientes_cidade")

In [0]:
## RELATÓRIO 6: CONTATO DE PACIENTES DE ALTO VALOR (TOP 50)

In [0]:
print("\n" + "="*60)
print("Criando: rpt_contato_pacientes_vip")
print("="*60)

rpt_contato_vip = df_consultas.alias("f") \
    .join(df_paciente.alias("p"), col("f.sk_paciente") == col("p.sk_paciente")) \
    .join(df_paciente_masked.alias("pm"), col("f.sk_paciente") == col("pm.sk_paciente")) \
    .groupBy(
        col("p.sk_paciente"),
        col("pm.nome_completo").alias("nome_masked"),
        col("pm.email").alias("email_masked"),
        col("p.cidade"),
        col("p.sexo")
    ) \
    .agg(
        count("f.id_consulta").alias("total_consultas"),
        sum("f.valor_total").alias("valor_total_gasto"),
        max(col("pm.silver_processed_at")).alias("ultima_atualizacao")
    ) \
    .filter(col("total_consultas") >= 3) \
    .orderBy(col("valor_total_gasto").desc()) \
    .limit(50)

result_contato_vip = save_to_gold(rpt_contato_vip, "rpt_contato_pacientes_vip")

In [0]:
## SUMÁRIO DE EXECUÇÃO

In [0]:
import pandas as pd
import builtins

results = [
    result_top_medicos,
    result_pacientes_gasto,
    result_medicos_clinica,
    result_historico,
    result_cidade,
    result_contato_vip
]

results_df = pd.DataFrame(results)
print("\n" + "="*80)
print("SUMÁRIO DE EXECUÇÃO - GOLD RELATÓRIOS COM IDENTIFICAÇÃO")
print("="*80)
display(results_df)

# Estatísticas
total_tables = len(results)
success_count = len([r for r in results if r['status'] == 'SUCCESS'])
failed_count = len([r for r in results if r['status'] == 'FAILED'])

if success_count > 0:
    total_records = builtins.sum([r.get('records', 0) for r in results if r['status'] == 'SUCCESS'])
    print(f"\nRelatórios criados: {success_count}/{total_tables}")
    print(f"Total de registros: {total_records:,}")

if failed_count > 0:
    print(f"\n⚠ ALERTA: {failed_count} relatórios falharam!")

print("\n" + "="*80)
print("RELATÓRIOS CRIADOS (COM DADOS MASCARADOS):")
print("="*80)
print("""
1. rpt_top_medicos_receita
   - Ranking de médicos por receita
   - Inclui: nome mascarado, CRM mascarado, especialidade

2. rpt_pacientes_alto_gasto
   - Top 100 pacientes por gasto total
   - Inclui: nome mascarado, email mascarado, cidade

3. rpt_medicos_por_clinica
   - Médicos atuantes por clínica
   - Inclui: nome médico mascarado, especialidade

4. rpt_historico_pacientes
   - Últimas 10 consultas por paciente
   - Inclui: nomes mascarados (paciente e médico)

5. rpt_medicos_pacientes_cidade
   - Distribuição por cidade/estado
   - Inclui: nomes mascarados

6. rpt_contato_pacientes_vip
   - Top 50 pacientes VIP para contato
   - Inclui: nome e email mascarados
""")

print("\n" + "="*80)
print("⚠ IMPORTANTE - LGPD:")
print("="*80)
print("""
- Todos os relatórios usam DADOS MASCARADOS
- CPF: ***.***.***-10
- Nome: João S.
- Email: j***@email.com
- CRM: ***456

- Para dados completos: Acesso restrito (DPO/Compliance)
- Logs de acesso devem ser mantidos
- Retenção conforme política LGPD
""")

print("="*80)
print("✓ Relatórios Gold concluídos!")
print("="*80)

In [0]:
%sql
select * from hive_metastore.healthcare_gold.rpt_medicos_por_clinica

In [0]:
spark.sql("SHOW TABLES IN healthcare_gold").show()